In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 256 kB in 4s (69.6 kB/s)
Reading 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-21 17:49:08--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.25MB/s    in 0.2s    

2022-08-21 17:49:09 (5.25 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [6]:
# total_votes count is equal to or greater than 20
vine_clean_df = vine_df.filter("total_votes >= 20")
vine_clean_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
vine_above50_df = vine_clean_df.withColumn("percent_helpful_votes", (vine_clean_df['helpful_votes']/vine_clean_df['total_votes'])*100).filter('percent_helpful_votes >= 50')
vine_above50_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|    61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                 60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|                 84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|    91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|    78.46153846153847|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
only showing top 5 rows



In [8]:
# Vine members only
from pyspark.sql.functions import col
vine_paid_df = vine_above50_df.filter(col("vine") == 'Y')
vine_paid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|    88.88888888888889|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|                100.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|    79.16666666666666|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|    84.61538461538461|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|    76.92307692307693|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
only showing top 5 rows



In [9]:
# Grouped DF based on star ratings by paid vine users
vine_paid_grp_df = vine_paid_df.groupBy("star_rating").agg({"star_rating":'count'}).withColumnRenamed("count(star_rating)", "paid_vine_star_count")
vine_paid_grp_df.show()

+-----------+--------------------+
|star_rating|paid_vine_star_count|
+-----------+--------------------+
|          1|                   1|
|          3|                  16|
|          5|                  48|
|          4|                  24|
|          2|                   5|
+-----------+--------------------+



In [10]:
# total votes from members with Vine membership
vine_paid_df.count()

94

In [11]:
# % 5 star votes among vine members
(48/vine_paid_df.count())*100

51.06382978723404

In [12]:
# Users without Vine membership
vine_notpaid_df = vine_above50_df.filter(col("vine") == 'N')
vine_notpaid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|    61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                 60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|                 84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|    91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|    78.46153846153847|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
only showing top 5 rows



In [13]:
# Grouped DF based on star ratings by users without vine
vine_notpaid_grp_df = vine_notpaid_df.groupBy("star_rating").agg({"star_rating":'count'}).withColumnRenamed("count(star_rating)", "not_paid_vine_star_count")
vine_notpaid_grp_df.show()

+-----------+------------------------+
|star_rating|not_paid_vine_star_count|
+-----------+------------------------+
|          1|                   10303|
|          3|                    4379|
|          5|                   15663|
|          4|                    6738|
|          2|                    3388|
+-----------+------------------------+



In [14]:
# total votes from members without Vine membership
vine_notpaid_df.count()

40471

In [15]:
# % 5 star votes among non-vine members
(15663/vine_notpaid_df.count())*100

38.701786464381904